In [ ]:
# package imports
import sklearn
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier

As mentioned in our project proposal, our baseline model will be a random forest classifier trained with scikit-learn.